In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModel

llama_causallm = LlamaForCausalLM.from_pretrained("NousResearch/Llama-2-7b-hf")
# llama_hf= AutoModel.from_pretrained("NousResearch/Llama-2-7b-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [2]:
llama_sd = llama_hf.state_dict()

for k, v in llama_sd.items():
    print(k, v.shape)

embed_tokens.weight torch.Size([32000, 4096])
layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
layers.0.self_attn.k_proj.weight torch.Size([4096, 4096])
layers.0.self_attn.v_proj.weight torch.Size([4096, 4096])
layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
layers.0.mlp.gate_proj.weight torch.Size([11008, 4096])
layers.0.mlp.up_proj.weight torch.Size([11008, 4096])
layers.0.mlp.down_proj.weight torch.Size([4096, 11008])
layers.0.input_layernorm.weight torch.Size([4096])
layers.0.post_attention_layernorm.weight torch.Size([4096])
layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
layers.1.self_attn.k_proj.weight torch.Size([4096, 4096])
layers.1.self_attn.v_proj.weight torch.Size([4096, 4096])
layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
layers.1.mlp.gate_proj.weight torch.Size([11008, 4096])
layers.1.mlp.up_proj.weight torch.Size([11008, 4096])
layers.1.mlp.down_proj.weight torch.Size([4096, 11008])
layers.1.input_layernorm.weight torch.Size([409

In [2]:
llama_causal_sd = llama_causallm.state_dict()

for k, v in llama_causal_sd.items():
    print(k, v.shape)

model.embed_tokens.weight torch.Size([32000, 4096])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.v_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.mlp.gate_proj.weight torch.Size([11008, 4096])
model.layers.0.mlp.up_proj.weight torch.Size([11008, 4096])
model.layers.0.mlp.down_proj.weight torch.Size([4096, 11008])
model.layers.0.input_layernorm.weight torch.Size([4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.v_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.1.mlp.gate_proj.weight torch.Size([11008, 4096])
model.layers.1.mlp.up_proj.weight torch.Size([11008, 4096])
model.l

In [23]:
import torch
import torch.nn as nn
from dataclasses import dataclass

@dataclass
class LlamaConfig:
    vocab: int = 32_000
    n_embd: int = 8
    intermediate_size: int = 11008
    
    n_layers: int = 32
    n_heads: int = 4
    n_kv_heads: int = 2

    batch_size: int = 2
    ctx_len: int = 3

config = LlamaConfig

In [24]:
tril = torch.tril(torch.ones((config.ctx_len, config.ctx_len), dtype=torch.long))
tril

tensor([[1, 0, 0],
        [1, 1, 0],
        [1, 1, 1]])

In [25]:
tril.shape

torch.Size([3, 3])

In [59]:
q = nn.Linear(config.n_embd, config.n_embd)
k = nn.Linear(config.n_embd, config.n_embd)

In [53]:
x = torch.rand((config.batch_size, config.ctx_len, config.n_embd))
x.shape

torch.Size([2, 3, 8])

In [54]:
B, T, C = x.shape
query = q(x)
print(query.shape)

query = query.view(B, T, config.n_heads, C // config.n_heads).transpose(1, 2)
print(query.shape)

torch.Size([2, 3, 8])
torch.Size([2, 4, 3, 2])


In [58]:
# k = k.unsqueeze(1).expand(-1, self.num_heads // self.kv_heads, -1, -1, -1)  # (B, n_heads // kv_heads, kv_heads, T, kv_head_dim)
query

tensor([[[[[-0.6331,  0.6262],
           [-0.1049,  0.4341],
           [-0.3236,  0.3640]],

          [[ 0.4240, -0.0697],
           [ 0.2945, -0.1620],
           [-0.0332, -0.1282]]],


         [[[ 0.2726,  0.0698],
           [ 0.5509, -0.0648],
           [ 0.3931, -0.1777]],

          [[ 0.0319,  0.6081],
           [-0.0959,  0.5124],
           [-0.1237,  0.7668]]]],



        [[[[-0.1491,  0.4938],
           [-0.1628,  0.2734],
           [-0.2886,  0.4039]],

          [[-0.1964,  0.3093],
           [ 0.0710,  0.0148],
           [-0.2560,  0.3555]]],


         [[[ 0.2366,  0.0188],
           [ 0.5626, -0.2082],
           [ 0.2074,  0.0888]],

          [[ 0.3507,  0.5721],
           [ 0.0201,  0.4847],
           [ 0.4706,  0.6150]]]]], grad_fn=<ViewBackward0>)

In [56]:
query = query.view(B, config.n_heads // config.n_kv_heads, config.n_kv_heads, T, C // config.n_heads)
print(query.shape)

torch.Size([2, 2, 2, 3, 2])


In [57]:
query[0]

tensor([[[[-0.6331,  0.6262],
          [-0.1049,  0.4341],
          [-0.3236,  0.3640]],

         [[ 0.4240, -0.0697],
          [ 0.2945, -0.1620],
          [-0.0332, -0.1282]]],


        [[[ 0.2726,  0.0698],
          [ 0.5509, -0.0648],
          [ 0.3931, -0.1777]],

         [[ 0.0319,  0.6081],
          [-0.0959,  0.5124],
          [-0.1237,  0.7668]]]], grad_fn=<SelectBackward0>)

In [61]:
key = k(x)
print(key.shape)
key = key.view(B, T, config.n_kv_heads, C // config.n_kv_heads).transpose(1, 2)
print(key.shape)

torch.Size([2, 3, 8])
torch.Size([2, 3, 2, 4])
